In [1]:
import pandas as pd
import pickle as pkl
import numpy as np

import sys
sys.path.append('../')
from utils import *

In [5]:
videos = pd.read_pickle('../../data/videos_raw_metadata')

In [4]:
watches = pd.read_pickle('../../data/user-level-data')
df = pd.read_pickle('../../data/datasets/raw-video-level-watches')
title_embeddings = pd.read_pickle('../../data/embeddings/title-autoencoded')
title_openai_embeddings = pd.read_pickle('../../data/embeddings/openai-title')
tag_embeddings = pd.read_pickle('../../data/embeddings/tag_embeddings.pkl')
topic_embeddings = pd.read_pickle('../../data/embeddings/topic_embeddings.pkl')

2023-08-25 12:48:41.746209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 12:48:42.217195: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-25 12:48:43.519926: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 443 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6
2023-08-25 12:48:43.520398: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46246 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:81:00.0, compute capability: 8.6


In [10]:
users = list(watches.keys())

In [11]:
user = users[9]
watch = watches[user]

In [7]:
def get_ids(watch):
    if watch['playing'] == None:
        return None
    if 'id' in watch['playing']:
        playing = watch['playing']['id']
    else:
        return None
    
    selected = watch['selected']

    suggesteds = []
    for s in watch['suggested']:
        if s == None:
            continue
        if 'id' in s:
            suggesteds.append(s['id'])

    w  = dict()
    w['playing'] = playing
    w['selected'] = selected
    w['suggesteds'] = suggesteds
    return w

In [8]:
watch_id = dict()
for user in users:
    watch_id[user] = []
    for watch in watches[user]:
        w = get_ids(watch)
        watch_id[user].append(w)

In [9]:
training_data = dict()
history_data = dict()
h = 0
t = 0
for user in users:
    training_data[user] = []
    history_data[user] = []

    for watch in watch_id[user]:
        if watch == None:
            continue
        if watch['playing'] == None:
            continue
        if watch['selected'] == None:
            history_data[user].append(watch['playing'])
            continue
        if watch['suggesteds'] == None:
            history_data[user].append(watch['playing'])
            continue
        if len(watch['suggesteds']) < 10:
            history_data[user].append(watch['playing'])
            continue

        training_data[user].append(watch)

In [ ]:
sessions = []

for user in users:
    for watch in training_data[user]:
        watch['user'] = user
        sessions.append(watch)

In [57]:
# pkl.dump(training_data, open('../../data/new_age/training_watches', 'wb'))
# pkl.dump(history_data, open('../../data/new_age/history_videos', 'wb'))
# pkl.dump(sessions, open('../../data/new_age/sessions', 'wb'))

In [1]:
# title_embeddings = pd.read_pickle('../../data/embeddings/title-autoencoded')
# title_openai_embeddings = pd.read_pickle('../../data/embeddings/openai-title')
tag_embeddings = pd.read_pickle('../../data/embeddings/tag_embeddings.pkl')
# topic_embeddings = pd.read_pickle('../../data/embeddings/topic_embeddings.pkl')

NameError: name 'pd' is not defined

In [15]:
vv = list(videos.keys())

In [20]:
tag_embeddings[vv[0]].shape

(300,)

In [21]:
v = vv[0]

In [25]:
videos[v]['snippet']['tags']

['good-day-atlanta',
 'news',
 'waga',
 'dana fowle',
 'fox 5 i-team',
 'mikel muffley',
 'muffley homes',
 'muffley']

In [3]:
import fasttext
import fasttext.util

# Download a pre-trained FastText model (change the parameters as needed)
model_path = fasttext.util.download_model('en', if_exists='ignore')  # 'en' for English

# Load the downloaded model
model = fasttext.load_model(model_path)

# Get embeddings for a word

def get_embeddings(word):
    return model.get_word_vector(word)

In [2]:
video_data = pkl.load(open('../../data/new_age/video_data', 'rb'))

2023-08-25 13:25:29.047366: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 13:25:29.781902: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-25 13:25:31.435699: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1337 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:01:00.0, compute capability: 8.6
2023-08-25 13:25:31.436307: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46246 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:81:00.0, compute capability: 8.6


In [7]:
tag_embs.shape

(300,)

In [14]:
from tqdm.notebook import tqdm

In [27]:
ids = videos.keys()
for video_id in tqdm(ids):
    video = videos[video_id]

    if 'tags' not in video['snippet']:
        tags = ['none']
    else:
        tags = video['snippet']['tags']
    


    tag_embs = []
    for tag in tags:
        tag_embs.append(get_embeddings(tag))

    tag_mean = np.mean(tag_embs, axis=0)

    tt = []
    for i in range(4):
        if i < len(tag_embs):
            tt.append(tag_embs[i])
        else:
            tt.append(np.zeros(300))
    tt.append(tag_mean)
    tt = np.array(tt)
    video_data[video_id]['bigger_tags'] = tt

  0%|          | 0/524537 [00:00<?, ?it/s]

In [28]:
pkl.dump(video_data, open('../../data/new_age/video_data', 'wb'))

In [ ]:
v = video_data[video_id]
v['title-short'].shape, v['title'].shape, v['tags'].shape, v['topics'].shape

dict_keys(['title-short', 'title', 'tags', 'topics'])

In [ ]:
history = dict()

for user in users:
    if user not in history_data:
        # empty vectors
        history[user]['title-short'] = np.zeros(512)
        history[user]['title'] = np.zeros(512)
        history[user]['tags'] = np.zeros(512)
        history[user]['topics'] = np.zeros(512)
        continue

    history[user] = dict()
    history[user]['title-short'] = []
    history[user]['title'] = []
    history[user]['tags'] = []
    history[user]['topics'] = []

    for video_id in history_data[user]:
        history[user]['title-short'].append(video_data[video_id]['title-short'])
        history[user]['title'].append(video_data[video_id]['title'])
        history[user]['tags'].append(video_data[video_id]['tags'])
        history[user]['topics'].append(video_data[video_id]['topics'])

    history[user]['title-short'] = np.mean(history[user]['title-short'])
    history[user]['title'] = np.mean(history[user]['title'])
    history[user]['tags'] = np.mean(history[user]['tags'])
    history[user]['topics'] = np.mean(history[user]['topics'])


In [16]:
content_path = '../../data/embeddings/channel embs/content.feather.zstd'
reddit_path = '../../data/embeddings/channel embs/reddit.feather.zstd'
recomm_path = '../../data/embeddings/channel embs/recomm.feather.zstd'

In [17]:
import pandas as pd
content = pd.read_feather(content_path)
reddit = pd.read_feather(reddit_path)
recom = pd.read_feather(recomm_path)

In [18]:
from collections import Counter
channels = []
for vid in videos:
    video = videos[vid]
    cid = video['snippet']['channelId']

NameError: name 'videos' is not defined